# Advanced Dataframes

## Agenda

1. Part I: Creating Dataframes
    - from Lists, Arrays, & Dictionaries
    - from PyDataset
    - from a SQL query
2. Exercises, Part I
3. Part II
    - Indexing and Subsetting
    - Aggregating
    - Merging & Joining
4. Exercises, Part II
5. Part III: Reshaping & Transposing
    - Reshaping
    - Transposing
10. Exercises III

In [42]:
# imports
import pandas as pd
import numpy as np

## Part I: Creating Dataframes

### From Lists

In [76]:
pd.DataFrame([[1,2,3],[4,5,6]])

,0,1,2
0,1,2,3
1,4,5,6


### From Arrays

In [79]:
a = np.array([[1,2,3],[4,5,6]])
pd.DataFrame(a, columns = ['a','b','c'])

,a,b,c
0,1,2,3
1,4,5,6


### From Dictionaries

In [81]:
pd.DataFrame({'a':[1,4],
             'b':[2,5],
             'c':[3,6]
             })

,a,b,c
0,1,2,3
1,4,5,6


### From PyDataset

In [46]:
from pydataset import data

### From SQL

We can use the read_sql method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted connection string.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a driver, a bit of software that handles the details of the database connection.

1. In order to connect to mysql, we'll install the pymysql driver packages by typing the following in your terminal: `python -m pip install pymysql'
1. **Add env.py to the repo's .gitignore file**
2. create a text file called env.py, and in your text file, write the following (filling in the corresponding info):

```python
host = '<ip_address>'
username = '<your_username>'
password = '<your_password>'
```

4. import the host, username, and password variables from the env.py file. Note that upon import, you use `env`, not `env.py`. **once you assign the string to the variable `url`, DO NOT print the value of url to your notebook. If you do this and push the notebook to github, your username and password will be visible to others.**

In [49]:

# create the url variable (string) with my username, password and host inserted

# write the SQL query to gather data, assign to variable 'query'

# read the data passing the function the query and the url string


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [81]:
# write a query in multiple lines:

## Exercises, Part I

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

2. cd into your exercises folder for this module and run echo env.py >> .gitignore

3. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

4. Use your function to obtain a connection to the employees database.

5. Once you have successfully run a query:
    - Intentionally make a typo in the database url. What kind of error message do you see?
    - Intentionally make an error in your SQL query. What does the error message look like?


6. Read the employees and titles tables into two separate DataFrames.

7. How many rows and columns do you have in each DataFrame? Is that what you expected?

8. Display the summary statistics for each DataFrame.

9. How many unique titles are in the titles DataFrame?

10. What is the oldest date in the to_date column?

11. What is the most recent date in the to_date column?

## Part II

### Indexing and Subsetting

- `[]`: subset rows using a boolean mask, or subset columns using a list of column names. 
- `.loc`: allows for subsetting rows and columns simultaneously, using the labels/names of rows/columns. 
- `.iloc`: allows for subsetting rows and columns simultaneously, using the index location or position of rows/columns. 

In [8]:
# Create list of values for names column.
import pandas as pd
import numpy as np

np.random.seed(123)

names = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas', 'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']


In [18]:
# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=101, size=len(names))
english_grades = np.random.randint(low=60, high=101, size=len(names))
reading_grades = np.random.randint(low=60, high=101, size=len(names))

classroom = np.random.choice(['A','B'], len(names))

In [19]:
# Construct the DataFrame using the above lists and arrays.


df = pd.DataFrame({'names': names,
                    'math': math_grades,
                      'english': english_grades,
                      'reading': reading_grades,
                      'classroom': classroom
                      })

Bracket: `[]`

In [20]:
# select 2 columns from the class
df[['names','classroom']]

,names,classroom
0,Sally,A
1,Jane,A
2,Suzie,B
3,Billy,A
4,Ada,A
5,John,A
6,Thomas,B
7,Marie,A
8,Albert,B
9,Richard,B


In [23]:
# I can pass a boolean Series to the indexing operator as a selector.
bools = df.names.str.startswith('A')

In [24]:
# then subset using brackets with the boolean series

`.loc`

- Select specific rows AND columns by index **label**. 
- The index label can be a number, but it can also be a string label. 
- It is **inclusive.**

`df.loc[row_indexer, column_indexer]`

In [25]:
# Select all the rows and a subset of columns; 
# notice the inclusive behavior of the indexing.
df.loc[:,'math':'reading'].head(5)

,math,english,reading
0,67,97,71
1,73,62,70
2,61,78,63
3,74,95,83
4,83,78,85


In [26]:
# I can use a boolean Series as a selector with .loc, too, 
# but I can choose rows and columns.
df.loc[bools, 'math':'reading']

,math,english,reading
4,83,78,85
8,84,63,74
11,93,88,83


`.iloc`

- Select specific rows and colums by index **position**. 
- It does **not** accept a boolean Series as a selector like .loc does. 
- It takes in integers representing index position
- It is **exclusive**.

`df.iloc[row_indexer, column_indexer]`

In [88]:
# select rows by integer position
# notice the exclusive behavior of the indexing

In [28]:
df.iloc[:4, 2:5]

,english,reading,classroom
0,97,71,A
1,62,70,A
2,78,63,B
3,95,83,A


### Aggregating

`.agg`

- Specify a way to aggregate a series of numerical values. 
- We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [29]:
df.reading.agg('min')

63

What happens if you run it on the entire dataframe, not just a series of a single column? 

In [31]:
# aggregate each column in the dataframe by taking the min value
df[['math','english','reading']].agg(['mean', 'min','max']).round(2)

,math,english,reading
mean,76.33,79.25,78.17
min,61.00,62.00,63.00
max,93.00,97.00,95.00


While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

`groupby`

- Creates a grouped object. 
- Then apply an aggregation on that object. 

For example, if we wanted to know the highest math grade from each classroom:

In [33]:
df.groupby('classroom').math.agg(['mean'])

,mean
classroom,
A,80.0
B,71.2


We can use `.agg` here to, to see multiple aggregations:

In [34]:
df.groupby('classroom')[['math','english']].agg(['mean',
                                                    'median',
                                                    'max',
                                                    'min'
                                                    ])

#nested columns

math                   english               
           mean median max min       mean median max min
classroom                                               
A          80.0   77.0  93  67  83.428571   87.0  97  62
B          71.2   65.0  84  61  73.400000   71.0  86  63

We can group by multiple columns as well. 

Let's compute the average reading grade for students who are passing english and those who are not passing english. 


In [37]:
# Create a boolean column named passing_english 
# based on the math column using np.where
df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

,names,math,english,reading,classroom,passing_math
0,Sally,67,97,71,A,failing
1,Jane,73,62,70,A,passing
2,Suzie,61,78,63,B,failing
3,Billy,74,95,83,A,passing
4,Ada,83,78,85,A,passing
5,John,93,87,87,A,passing
6,Thomas,63,69,63,B,failing
7,Marie,77,77,95,A,passing
8,Albert,84,63,74,B,passing
9,Richard,83,71,81,B,passing


In [40]:
# Group by the combination of our new feature, passing_english,
# and the classroom.
# aggregate by calculating average reading grade
grade_groups = df.groupby(['classroom','passing_math']).reading.agg(['mean','count'])

In [42]:
# clean up the column names to make the data more transparent
grade_groups.index
grade_groups.columns = ['avg_reading_grade', 'number_of_students']
grade_groups

avg_reading_grade  number_of_students
classroom passing_math                                       
A         failing               71.000000                   1
          passing               83.833333                   6
B         failing               69.666667                   3
          passing               77.500000                   2

`.transform`

When you want to add an aggregate value, like the average math grades for each classroom, back to the original dataframe. 

In [45]:
df.groupby('classroom').math.agg('mean')

classroom
A    80.0
B    71.2
Name: math, dtype: float64

In [51]:
df.assign(avg_math_by_classroom = df.groupby('classroom').math.transform('mean'))
#transform works to spread it across the dataframe

,names,math,english,reading,classroom,passing_math,avg_math_by_classroom
0,Sally,67,97,71,A,failing,80.0
1,Jane,73,62,70,A,passing,80.0
2,Suzie,61,78,63,B,failing,71.2
3,Billy,74,95,83,A,passing,80.0
4,Ada,83,78,85,A,passing,80.0
5,John,93,87,87,A,passing,80.0
6,Thomas,63,69,63,B,failing,71.2
7,Marie,77,77,95,A,passing,80.0
8,Albert,84,63,74,B,passing,71.2
9,Richard,83,71,81,B,passing,71.2


In [52]:
df

,names,math,english,reading,classroom,passing_math
0,Sally,67,97,71,A,failing
1,Jane,73,62,70,A,passing
2,Suzie,61,78,63,B,failing
3,Billy,74,95,83,A,passing
4,Ada,83,78,85,A,passing
5,John,93,87,87,A,passing
6,Thomas,63,69,63,B,failing
7,Marie,77,77,95,A,passing
8,Albert,84,63,74,B,passing
9,Richard,83,71,81,B,passing


`.describe` 

Can be used with `groupby` also. 

In [56]:
df.groupby('classroom').describe().T

classroom              A          B
math    count   7.000000   5.000000
        mean   80.000000  71.200000
        std    10.082989  11.322544
        min    67.000000  61.000000
        25%    73.500000  63.000000
        50%    77.000000  65.000000
        75%    88.000000  83.000000
        max    93.000000  84.000000
english count   7.000000   5.000000
        mean   83.428571  73.400000
        std    12.122392   8.848729
        min    62.000000  63.000000
        25%    77.500000  69.000000
        50%    87.000000  71.000000
        75%    91.500000  78.000000
        max    97.000000  86.000000
reading count   7.000000   5.000000
        mean   82.000000  72.800000
        std     8.850612   9.549869
        min    70.000000  63.000000
        25%    77.000000  63.000000
        50%    83.000000  74.000000
        75%    86.000000  81.000000
        max    95.000000  83.000000

### Merging & Joining

Ways to combine dataframes together

`pd.concat`

- This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis
- Row-wise: axis=0, means your new data frame will be longer with more rows. 
- Column-wise: axis=1, means your new dataframe will be wider with more columns. 
- When your list contains at least one DataFrame, a DataFrame is returned.
- When concatenating only Series objects row-wise, axis=0, a Series is returned.
- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

default parameters: 

`pd.concat(objs_in_list, axis=0, join='outer')`

In [92]:
# concatenate 2 dataframes by passing a list of the 2 dfs, and 
# indicating whether to concatenate along the rows or the columns

In [57]:
df.shape

(12, 6)

In [67]:
df1 = pd.DataFrame({'a': [1,2,3]})
df2 = pd.DataFrame({'a': [98,6,5]})
pd.concat([df1,df2], axis=0)

,a
0,1
1,2
2,3
0,98
1,6
2,5


`.merge`

- Similar to a SQL join.
- first dataframe is the 'left' table. 
- `how`: Type of merge to be performed.
    - `how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.
    - `how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.
    - `how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
    - `how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.


In [70]:
# Create the users DataFrame.
users = pd.DataFrame({
    'id':[1,2,3,4,5,6],
    'name':['bob','joe','sally','adam','jorge', 'mike'],
    'role_id':[1,2,3,3,np.nan,np.nan]
    
    
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jorge,NaN
5,6,mike,NaN


In [75]:
# Create the roles DataFrame
roles = pd.DataFrame({
    'id': [1,2,3,4],
    'name':['admin','author','reviewer','commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


- `left_on` and `right_on`: indicate the columns that are the keys used to merge the dataframes together.

- `indicator`=`True` will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [73]:
# Perform an outer join specifying the left and right DataFrame keys.
users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jorge,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


- suffix of `_x` to any columns in the left dataframe that are duplicated, 
- suffix of `_y` to any columns in the right dataframe that are duplicated. 
- can clean up column names using **method chaining**.

In [74]:
(users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)
             .drop(columns = 'role_id')
            .rename(columns={'id_x':'id',
                            'name_x':'employee',
                             'id_y':'role_id',
                             'name_y':'role'}))

,id,employee,role_id,role,_merge
0,1.0,bob,1.0,admin,both
1,2.0,joe,2.0,author,both
2,3.0,sally,3.0,reviewer,both
3,4.0,adam,3.0,reviewer,both
4,5.0,jorge,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,left_only
6,NaN,NaN,4.0,commenter,right_only


## Part 3: Reshaping and Transposing

1. Reshaping: pd.crosstab, .pivot_table
2. Transposing

### Reshaping

`pd.crosstab`

Count the number of students passing math in each classroom (student grades df):

Add subtotals using the `margins` argument:

View percentages instead of actual values using the `normalize` argument. 

`.pivot_table`

Like an excel pivot table

Supply 4 pieces of info to arguments: 
- which values will make up the rows, the `index`
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`) (default is `mean`). 

Example: Calculate the average math grade for the combination of `classroom` and `passing_math` status. 

Create a new dataframe: orders at a restaurant. 

`.map`

Use a dictionary to calculate the total price for an order. Then save the calculations to a new column named `bill`. 

In [96]:
# create a dictionary of prices for drinks and meals

In [97]:
# match the values in the drink and meal columns with the values 
# in the prices and perform the specificied calculations. 
# save to a new column, bill

In [98]:
# use crosstab to look at how many orders have each combination of
# meal and drink

In [99]:
# find the average bill for each combination using pivot_table

In [100]:
# find the average bill for each combination using groupby

Transposing

Swapping rows and columns